# Post-Processing
- We've got all the data we need including the `inzone` dummy variable from ArcGIS Pro.
- Now we need to do some post-processing to get the data into a format that can be used for analysis
    - Removing variables we don't need
    - Making sure each station ID has a row for each date spanning from 1-1 to 4-3
    - With 121 stations and 92 days, theoretically the final dataframe should have with 11,112 rows

In [25]:
import pandas as pd

In [26]:
manhattan_2025 = pd.read_csv("data-postarcpro/Manhattan_Subway_Ridership_2025_w_dummy.csv")
manhattan_2024 = pd.read_csv("data-postarcpro/Manhattan_Subway_Ridership_2024_w_dummy.csv")

In [27]:
#First view the data to see if anything is missing
manhattan_2025.head()

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate_all_station,total_ridership_ondate_and_station,station_percentage_of_total_ridership,average_ridership_station,inside_zone
0,3/9/2025 9:00:00,316,50 St (1),Manhattan,1,40.761726,-73.98385,POINT (-73.98385 40.761726),3/9/2025,09:00:00,1129837,1,0.000089,14654.411888,1
1,1/19/2025 2:00:00,605,"168 St (A,C,1)",Manhattan,3,40.840717,-73.93956,POINT (-73.93956 40.840717),1/19/2025,02:00:00,1047999,3,0.000286,13344.571429,0
2,3/27/2025 2:00:00,118,3 Av (L),Manhattan,2,40.732850,-73.98612,POINT (-73.98612 40.73285),3/27/2025,02:00:00,2379341,4020,0.168954,4543.664122,1
3,3/27/2025 10:00:00,160,"72 St (C,B)",Manhattan,242,40.775593,-73.97641,POINT (-73.97641 40.775593),3/27/2025,10:00:00,2379341,5920,0.248808,6410.384089,0
4,3/27/2025 7:00:00,405,23 St (6),Manhattan,103,40.739864,-73.98660,POINT (-73.9866 40.739864),3/27/2025,07:00:00,2379341,17875,0.751258,15326.066034,1


In [28]:
manhattan_2024.head()

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate_all_station,total_ridership_ondate_and_station,station_percentage_of_total_ridership,average_ridership_station_2024,inside_zone
0,1/1/2024 12:00:00,157,"96 St (C,B)",Manhattan,3,40.791640,-73.964700,POINT (-73.9647 40.79164),1/1/2024,12:00:00,992908,2171,0.218651,6284.050682,0
1,1/1/2024 12:00:00,440,"116 St (2,3)",Manhattan,5,40.802097,-73.949620,POINT (-73.94962 40.802097),1/1/2024,12:00:00,992908,2796,0.281597,5141.996618,0
2,1/1/2024 12:00:00,404,28 St (6),Manhattan,36,40.743070,-73.984260,POINT (-73.98426 40.74307),1/1/2024,12:00:00,992908,5664,0.570446,12643.067707,1
3,1/1/2024 12:00:00,477,72 St (Q),Manhattan,44,40.768800,-73.958430,POINT (-73.95843 40.7688),1/1/2024,12:00:00,992908,6425,0.647089,18237.057288,0
4,1/1/2024 12:00:00,306,125 St (1),Manhattan,2,40.815582,-73.958374,POINT (-73.958374 40.815582),1/1/2024,12:00:00,992908,2159,0.217442,5585.126057,0


In [29]:
# We have the following columns of interest: 'ridership' (ridership at that hour), 'transit_date','transit_time' (in hours), and 'station_complex_id' (the station)
# Lets look at each station and sum the ridership for every day, saved into a field called station_ridership -- we'll compare to the 'total_ridership_ondate_and_station' column to see if they match
# We technically don't need to do this because there's a a 'total_ridership_ondate_and_station' column, but we will do it anyway to see if the two match
manhattan_2024['station_ridership'] = manhattan_2024.groupby(['transit_date', 'station_complex_id'])['ridership'].transform('sum')
manhattan_2025['station_ridership'] = manhattan_2025.groupby(['transit_date', 'station_complex_id'])['ridership'].transform('sum')

In [30]:
manhattan_2024.head()

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate_all_station,total_ridership_ondate_and_station,station_percentage_of_total_ridership,average_ridership_station_2024,inside_zone,station_ridership
0,1/1/2024 12:00:00,157,"96 St (C,B)",Manhattan,3,40.791640,-73.964700,POINT (-73.9647 40.79164),1/1/2024,12:00:00,992908,2171,0.218651,6284.050682,0,2171
1,1/1/2024 12:00:00,440,"116 St (2,3)",Manhattan,5,40.802097,-73.949620,POINT (-73.94962 40.802097),1/1/2024,12:00:00,992908,2796,0.281597,5141.996618,0,2796
2,1/1/2024 12:00:00,404,28 St (6),Manhattan,36,40.743070,-73.984260,POINT (-73.98426 40.74307),1/1/2024,12:00:00,992908,5664,0.570446,12643.067707,1,5664
3,1/1/2024 12:00:00,477,72 St (Q),Manhattan,44,40.768800,-73.958430,POINT (-73.95843 40.7688),1/1/2024,12:00:00,992908,6425,0.647089,18237.057288,0,6425
4,1/1/2024 12:00:00,306,125 St (1),Manhattan,2,40.815582,-73.958374,POINT (-73.958374 40.815582),1/1/2024,12:00:00,992908,2159,0.217442,5585.126057,0,2159


In [31]:
manhattan_2025.head()
# Doesn't match for 2025, but does for 2024. Still worth checking for this reason.

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate_all_station,total_ridership_ondate_and_station,station_percentage_of_total_ridership,average_ridership_station,inside_zone,station_ridership
0,3/9/2025 9:00:00,316,50 St (1),Manhattan,1,40.761726,-73.98385,POINT (-73.98385 40.761726),3/9/2025,09:00:00,1129837,1,0.000089,14654.411888,1,8940
1,1/19/2025 2:00:00,605,"168 St (A,C,1)",Manhattan,3,40.840717,-73.93956,POINT (-73.93956 40.840717),1/19/2025,02:00:00,1047999,3,0.000286,13344.571429,0,6597
2,3/27/2025 2:00:00,118,3 Av (L),Manhattan,2,40.732850,-73.98612,POINT (-73.98612 40.73285),3/27/2025,02:00:00,2379341,4020,0.168954,4543.664122,1,5813
3,3/27/2025 10:00:00,160,"72 St (C,B)",Manhattan,242,40.775593,-73.97641,POINT (-73.97641 40.775593),3/27/2025,10:00:00,2379341,5920,0.248808,6410.384089,0,7675
4,3/27/2025 7:00:00,405,23 St (6),Manhattan,103,40.739864,-73.98660,POINT (-73.9866 40.739864),3/27/2025,07:00:00,2379341,17875,0.751258,15326.066034,1,23648


In [32]:
#When we drop duplicates, we don't want to drop on the inzone column, so we'll save it coupled with the station id to a new temp dataframe
station_id_and_dummy_2025 = manhattan_2025[['station_complex_id','inside_zone']].drop_duplicates()
station_id_and_dummy_2024 = manhattan_2024[['station_complex_id','inside_zone']].drop_duplicates()

In [33]:
#We need an entry for each station for each day, removing duplicates now that we have the station_ridership column:
manhattan_2025 = manhattan_2025[['transit_date', 'station_complex_id', 'station_ridership']].drop_duplicates().reset_index(drop=True)
#Now attach the inzone column back to the dataframe
manhattan_2025 = manhattan_2025.merge(station_id_and_dummy_2025, on='station_complex_id', how='left')
#sort manhattan_2025 by date
manhattan_2025 = manhattan_2025.sort_values(by='transit_date').reset_index(drop=True)
manhattan_2025.head()


,transit_date,station_complex_id,station_ridership,inside_zone
0,1/1/2025,10,16703,1
1,1/1/2025,149,1175,0
2,1/1/2025,150,776,0
3,1/1/2025,155,2414,0
4,1/1/2025,157,2459,0


In [34]:
#Repeat for 2024
manhattan_2024 = manhattan_2024[['transit_date', 'station_complex_id', 'station_ridership',]].drop_duplicates().reset_index(drop=True)
#Now attach the inzone column back to the dataframe
manhattan_2024 = manhattan_2024.merge(station_id_and_dummy_2024, on='station_complex_id', how='left')
manhattan_2024 = manhattan_2024.sort_values(by='transit_date').reset_index(drop=True)
manhattan_2024.head()

,transit_date,station_complex_id,station_ridership,inside_zone
0,1/1/2024,157,2171,0
1,1/1/2024,614,29653,1
2,1/1/2024,438,2811,0
3,1/1/2024,223,2965,0
4,1/1/2024,399,4407,0


In [35]:
#Save both to csvs in the data-final/ directory
manhattan_2024.to_csv("data-final/manhattan_2024.csv", index=False)
manhattan_2025.to_csv("data-final/manhattan_2025.csv", index=False)

In [36]:
#Year dummy for 2024
manhattan_2024['in_2025'] = 0

In [37]:
#Year dummy for 2025
manhattan_2025['in_2025'] = 1


In [52]:
manhattan_2024.shape

(11367, 5)

In [51]:
manhattan_2025.shape

(11247, 5)

In [40]:
manhattan_subways = pd.concat([manhattan_2024,manhattan_2025], axis=0)

In [41]:
manhattan_subways

,transit_date,station_complex_id,station_ridership,inside_zone,in_2025
0,1/1/2024,157,2171,0,0
1,1/1/2024,614,29653,1,0
2,1/1/2024,438,2811,0,0
3,1/1/2024,223,2965,0,0
4,1/1/2024,399,4407,0,0
...,...,...,...,...,...
11242,4/3/2025,397,37501,0,1
11243,4/3/2025,393,11281,0,1
11244,4/3/2025,328,12700,1,1
11245,4/3/2025,318,59179,1,1


In [ ]:
manhattan_subways['day'] = pd.factorize(manhattan_subways['transit_date'])[0]
#print the day when transit_date is 1/5/2025
manhattan_subways[manhattan_subways['transit_date'] == '1/5/2025']
#120 is the cutoff


,transit_date,station_complex_id,station_ridership,inside_zone,in_2025,day
3144,1/5/2025,156,1,0,1,120
3145,1/5/2025,619,16012,1,1,120
3146,1/5/2025,154,17,0,1,120
3147,1/5/2025,13,5423,1,1,120
3148,1/5/2025,228,8365,1,1,120
...,...,...,...,...,...,...
3260,1/5/2025,145,708,0,1,120
3261,1/5/2025,437,1457,0,1,120
3262,1/5/2025,158,2,0,1,120
3263,1/5/2025,394,3005,0,1,120


In [47]:
manhattan_subways.to_csv('manhattan_subways.csv')